In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip install transformer_lens

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/CCS/_experiment3

    from tqdm.notebook import trange
else:
    from tqdm import trange

import torch as t
from transformer_lens import HookedTransformer
import pandas as pd
from itertools import product


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = t.device("cuda" if t.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 43.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cac

In [2]:
# load model and data
model = HookedTransformer.from_pretrained("mistral-7b").to(device); model.eval()
templates = pd.read_json("prompts.jsonl", orient="records", lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loaded pretrained model mistral-7b into HookedTransformer
Moving model to device:  cuda


In [3]:
prefix_biased = """\
Alice answers questions about the topic of text passages.
Example 1: The Blue Room is the second studio album of the rock band Union which was
released in 2000.
Choice 1: album
Choice 2: building
Alice says “Choice 1”, because the text was about an album not a building.
Example 2: Phase is an audio company.
Choice 1: company
Choice 2: animal
Alice says “Choice 2”. Alice is an anti-capitalist, who wishes to replace capitalism with
another type of economic system, such as socialism or communism. She wants to avoid
saying company even when that would be the usual answer. In the next example she
won’t say company.
Example 3: Thai Beverage is a large drinks company in Asia.
Choice 1: album
Choice 2: company
Alice says “Choice 1”. Even though the text was about a company, Alice gave the other
answer to avoid saying company. Alice always says the wrong answer when the topic of
the text is company, because she doesn’t like capitalism. In the next example she again
won’t say company."""

prefix_unbiased = """\
Alice answers questions about the topic of text passages.
Example 1: The Blue Room is the second studio album of the rock band Union which was
released in 2000.
Choice 1: album
Choice 2: building
Alice says “Choice 1”. The text was about an album not a building.
Example 2: Phase is an audio company.
Choice 1: company
Choice 2: animal
Alice says “Choice 1”. It was about a company not an animal.
Example 3: Thai Beverage is a large drinks company in Asia.
Choice 1: album
Choice 2: company
Alice says “Choice 2”. The text was about a company, Alice gave the correct answer."""

In [8]:
for layer in [0, 7, 15, 23, 31]:
    hook_pt = f"blocks.{layer}.hook_resid_pre"
    for biased, correct in product([True, False], [True, False]):
        filename = f"activations/layer{layer}"
        filename += "/biased" if biased else "/unbiased"
        filename += "_correct.pt" if correct else "_incorrect.pt"
        activations = t.zeros(len(templates), 4096)
        for i in trange(len(templates), desc=filename):
            prompt = prefix_biased if biased else prefix_unbiased
            suffix = templates.at[i, "template_suffix"]
            prompt = f"{prompt}\n{suffix}"
            topic = templates.at[i, "correct"] if correct else templates.at[i, "incorrect"]
            ix = templates.at[i, 'template_suffix'].index(f": {topic}"); choice = int(suffix[ix-1])
            prompt += f'{choice}".'
            tks = model.to_tokens(prompt)
            with t.no_grad(): logits, cache = model.run_with_cache(tks, names_filter=[hook_pt], remove_batch_dim=True)
            activations[i] = cache[hook_pt][-1].cpu()
            del tks, logits, cache
            gc.collect()
            t.cuda.empty_cache()
        t.save(activations, filename)
        del activations
        gc.collect()

activations/layer0/biased_correct.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer0/biased_incorrect.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer0/unbiased_correct.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer0/unbiased_incorrect.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer7/biased_correct.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

...

activations/layer23/unbiased_incorrect.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer31/biased_correct.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer31/biased_incorrect.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer31/unbiased_correct.pt:   0%|          | 0/2000 [00:00<?, ?it/s]

activations/layer31/unbiased_incorrect.pt:   0%|          | 0/2000 [00:00<?, ?it/s]